In [ ]:
!git clone https://github.com/LINKS-Foundation-CPE/Master-QCC-2022-2023.git

In [ ]:
!pip install dwave_qbsolv
!pip install scipy

In [ ]:
%cd /content/Master-QCC-2022-2023/

In [1]:
from itertools import combinations
import numpy as np
import random
from dwave_qbsolv import QBSolv

In [2]:
# num_aminoacids = 50
# hydrophobic_pos = [2, 4, 5, 6, 11, 12, 17, 20, 21, 25, 27, 28, 30, 31, 33, 37, 44 ,46]

#initialize variables

random.seed(1)
num_aminoacids = 30
hydrophobic_pos = random.sample(range(1,num_aminoacids+1), 10)
hydrophobic_pos=np.sort(hydrophobic_pos)
all_pairs =list(combinations(hydrophobic_pos, 2))
pairs_dict = {}
rev_pairs_dict = {}
index_pair = 0
gap_dict={}
pairs=[]
for pair in all_pairs:
    if (pair[1]-pair[0]-1)%2==0 and (pair[1]-pair[0]-1)!=0:
        pairs.append(pair)
        pairs_dict[pair]=index_pair
        rev_pairs_dict[index_pair]=pair
        index_pair+=1
        gap=pair[0]+pair[1]-1
        if (gap)%2==0:
            if gap/2 in gap_dict:
                gap_dict[gap/2].append(pair)
            else:
                gap_dict[gap/2]=[pair]

dimQ = len(pairs_dict)

Q = np.zeros((dimQ, dimQ))
P = 100

In [3]:
#QUBO matrix generation

# objective function
Q[np.diag_indices_from(Q)]=-1
for pair in pairs:
    i = pair[0]
    j = pair[1]
    k_array = list(range(i, j))
    k_del = (i+j-1)/2
    if k_del in k_array:
        k_array.remove(k_del)
    for k_val in k_array:
        if k_val in gap_dict:
            pairs_to_penalize = gap_dict[k_val]
            for p in pairs_to_penalize:
                index_1 = pairs_dict[p]
                index_2 = pairs_dict[pair]
                if index_1 < index_2:
                    Q[index_1, index_2]=P
                else:
                    Q[index_2, index_1]=P
        # elif k_val in other_gaps:
        #     index_1 = pairs_dict[pair]
        #     index_2 = other_gaps[k_val]
        #     Q[index_1, index_2]=P
Q_dict={}
for i in range(dimQ):
    for j in range(dimQ):
        Q_dict[(i,j)] = Q[i,j]

In [4]:

#Solve QUBO
response = QBSolv().sample_qubo(Q_dict, timeout=30, num_repeats=100)
all_energies = response.data_vectors['energy']
min_energy = response.data_vectors['energy'].min()
num_sol = len(all_energies)
sol_str = 'Number of matching found: {}\n\n'.format(-min_energy)
for sol in range(num_sol):    
    if all_energies[sol]==min_energy:
        sol_str += 'Solution {}\n'.format(sol)
        print()
        item = response.samples()[sol]
        for var in item:
            if item[var]==1 and var in rev_pairs_dict:
                matching= rev_pairs_dict[var]
                fold = (matching[0]+matching[1]-1)/2
                sol_str+='matching {} with folding at aminoacid {}\n'.format(matching, fold)

/var/folders/qp/68cmb73d6897nh_qdb2y19fr0000gn/T/ipykernel_7021/1404573121.py:2: DeprecationWarning: QBSolv is deprecated as of the end of 2021 and support will be discontinued after March 2022. Please update your code to use dwave-hybrid or Leap's quantum-classical hybrid solvers instead.
  response = QBSolv().sample_qubo(Q_dict, timeout=30, num_repeats=100)


In [5]:
with open('solutions.txt', 'w') as fp:
    fp.write(sol_str)

print('Finished QBSolv')

Finished QBSolv
